## A. PHƯƠNG PHÁP TÌM GÓC HARRIS

Hoàn thiện file 'Eg1_Harris_Conner_Detect_numpy_cv_FIXED.ipynb' và file 'Eg2_Harris_Conner_Detect_numpy_cv_FIXED.ipynb' được đính k

## B. SCALE INVARIANT FEATURE TRANSFORMATION: HARRIS - LAPLACIAN

In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

# Constants
CONSTANT_K = 2 ** (1.0 / 3)
CONTRAST_THRESHOLD = 0.03
CURVATURE_THRESHOLD = 10
RATIO_THRESHOLD = (CURVATURE_THRESHOLD + 1) ** 2 / CURVATURE_THRESHOLD

# Assume convolve function from part A (using cv2.filter2D for 2D convolution)
def convolve(img, kernel):
    return cv2.filter2D(img, ddepth=-1, kernel=kernel)

# (i) Laplacian kernel 3x3
laplacianKernel = np.array([[-1, -1, -1],
                            [-1, 8, -1],
                            [-1, -1, -1]], np.int32)

# (ii) LoG kernel 5x5 for base sigma
log5Kernel = np.array([[0, 0, 1, 0, 0],
                       [0, 1, 2, 1, 0],
                       [1, 2, -16, 2, 1],
                       [0, 1, 2, 1, 0],
                       [0, 0, 1, 0, 0]], np.int32)

# (iii) Gaussian generator (for creating Gaussian kernels if needed)
def gaussianGenerator(size=0, sigma=0):
    if size == 0 and sigma != 0:
        size = 2 * math.ceil(3 * sigma) + 1
    if size != 0 and sigma == 0:
        sigma = (size // 2) / 2
    # Gaussian in X direction
    gaussX = gaussianXFunction(size, sigma)
    # Gaussian in Y is transpose
    gaussY = gaussX.transpose()
    # Normalize
    sumX = np.sum(gaussX)
    sumY = np.sum(gaussY)
    gaussX = gaussX / sumX
    gaussY = gaussY / sumY
    return gaussX, gaussY

def gaussianXFunction(size, sigma):
    minX = -size / 2
    maxX = size / 2
    step = 1
    a_range = np.arange(minX, maxX - step + 1.0, step)
    b_range = np.arange(minX + step, maxX + 1.0, step)
    # Vectorized integrand for Gaussian
    def gaussX_int(f, sigma, a, b):
        # Approximate integral of Gaussian from a to b
        return (1 / (sigma * math.sqrt(2 * np.pi))) * (math.exp(-a**2 / (2 * sigma**2)) - math.exp(-b**2 / (2 * sigma**2)))
    vec_gaussX_int = np.vectorize(gaussX_int)
    gaussX = vec_gaussX_int(lambda x: x, sigma, a_range, b_range)
    return gaussX.reshape(-1, 1)

# (iv) LoG function
def LoG_function(sigma):
    ksize = 2 * math.ceil(3 * sigma) + 1
    radious = ksize // 2
    x, y = np.ogrid[-radious:radious+1, -radious:radious+1]
    ex = np.exp(-(x**2) / (2.0*sigma*sigma))
    ey = np.exp(-(y**2) / (2.0*sigma*sigma))
    LoG = (-2*sigma**2 + (x*x + y*y)) * (ex*ey) * (1.0 / (2*np.pi*sigma**4))
    return LoG

# (v) Detect by LoG
def detectByLoG(logKernel, img):
    logImg = convolve(img, logKernel)
    return logImg

# (vi) Create LoG pyramid
def create_log_pyramid(img, initSigma=1.0, no_scale_lv=9, CONSTANT_K=CONSTANT_K):
    pyramid = []
    next_sigma = lambda k, sigma: k * sigma
    for i in range(no_scale_lv):
        curSigma = next_sigma(CONSTANT_K ** i, initSigma)
        myfilter = LoG_function(curSigma)
        logImg = detectByLoG(myfilter, img)
        logImg = np.pad(logImg, (1, 1), mode='constant')
        # Normalize before squaring to prevent overflow
        logImg = logImg.astype(np.float64)
        logImg = logImg / (np.max(np.abs(logImg)) + 1e-8)
        logImg = np.square(logImg)
        pyramid.append(logImg)
    return np.array(pyramid)

# (vii) Detect blobs by LoG (original with curvature ratio)
def detectBlobByLoG_original(img, initSigma=1.0, no_scale_lv=9, CONSTANT_K=CONSTANT_K, CONTRAST_THRESHOLD=CONTRAST_THRESHOLD, ratio_threshold=RATIO_THRESHOLD):
    LoG_pyramid = create_log_pyramid(img, initSigma, no_scale_lv, CONSTANT_K)
    keypoints = []
    iH, iW = img.shape
    xidx = np.array([-1, -1, -1, 0, 0, 1, 1, 1, 0])
    yidx = np.array([-1, 0, 1, -1, 1, -1, 0, 1, 0])
    for iz in range(1, no_scale_lv - 1):
        cur_img = LoG_pyramid[iz]
        prev_img = LoG_pyramid[iz - 1]
        next_img = LoG_pyramid[iz + 1]
        for x in range(1, iH):
            for y in range(1, iW):
                if (np.all(cur_img[x][y] >= cur_img[x + xidx[:-1], y + yidx[:-1]]) and
                    np.all(cur_img[x][y] >= prev_img[x + xidx, y + yidx]) and
                    np.all(cur_img[x][y] >= next_img[x + xidx, y + yidx])):
                    if cur_img[x][y] < CONTRAST_THRESHOLD:
                        continue
                    dxx = cur_img[x][y - 1] + cur_img[x][y + 1] - 2 * cur_img[x][y]
                    dyy = cur_img[x - 1][y] + cur_img[x + 1][y] - 2 * cur_img[x][y]
                    dxy = (cur_img[x - 1][y - 1] + cur_img[x + 1][y + 1] -
                           cur_img[x - 1][y + 1] -
                           cur_img[x + 1][y - 1]) / 4.0
                    trH = dxx + dyy
                    detH = dxx * dyy - dxy ** 2
                    curvature_ratio = np.nan_to_num(trH ** 2 / detH)
                    if curvature_ratio <= ratio_threshold:
                        keypoints.append((x - 1, y - 1, CONSTANT_K ** iz * initSigma))
    return keypoints

# Modified to use Harris R instead of curvature ratio
def detectBlobByLoG(img, initSigma=1.0, no_scale_lv=9, CONSTANT_K=CONSTANT_K, CONTRAST_THRESHOLD=CONTRAST_THRESHOLD, k=0.05, R_threshold=0):
    LoG_pyramid = create_log_pyramid(img, initSigma, no_scale_lv, CONSTANT_K)
    keypoints = []
    iH, iW = img.shape
    xidx = np.array([-1, -1, -1, 0, 0, 1, 1, 1, 0])
    yidx = np.array([-1, 0, 1, -1, 1, -1, 0, 1, 0])

    for iz in range(1, no_scale_lv - 1):
        cur_img = LoG_pyramid[iz].astype(np.float64)
        prev_img = LoG_pyramid[iz - 1].astype(np.float64)
        next_img = LoG_pyramid[iz + 1].astype(np.float64)

        for x in range(1, iH):
            for y in range(1, iW):
                if (np.all(cur_img[x][y] >= cur_img[x + xidx[:-1], y + yidx[:-1]]) and
                    np.all(cur_img[x][y] >= prev_img[x + xidx, y + yidx]) and
                    np.all(cur_img[x][y] >= next_img[x + xidx, y + yidx])):
                    if cur_img[x][y] < CONTRAST_THRESHOLD:
                        continue

                    # Use safe arithmetic with overflow checking
                    try:
                        val_center = float(cur_img[x][y])
                        val_left = float(cur_img[x][y - 1]) if y > 0 else val_center
                        val_right = float(cur_img[x][y + 1]) if y < iW - 1 else val_center
                        val_up = float(cur_img[x - 1][y]) if x > 0 else val_center
                        val_down = float(cur_img[x + 1][y]) if x < iH - 1 else val_center

                        # Compute second derivatives safely
                        dxx = val_left + val_right - 2 * val_center
                        dyy = val_up + val_down - 2 * val_center

                        # Compute mixed derivative safely
                        if x > 0 and y > 0 and x < iH - 1 and y < iW - 1:
                            val_ul = float(cur_img[x - 1][y - 1])
                            val_ur = float(cur_img[x - 1][y + 1])
                            val_dl = float(cur_img[x + 1][y - 1])
                            val_dr = float(cur_img[x + 1][y + 1])
                            dxy = (val_ul + val_dr - val_ur - val_dl) / 4.0
                        else:
                            dxy = 0.0

                        # Check for overflow/underflow
                        if not (np.isfinite(dxx) and np.isfinite(dyy) and np.isfinite(dxy)):
                            continue

                        trH = dxx + dyy
                        detH = dxx * dyy - dxy * dxy

                        if not (np.isfinite(trH) and np.isfinite(detH)):
                            continue

                        R = detH - k * (trH * trH)

                        if np.isfinite(R) and R > R_threshold:
                            keypoints.append((x - 1, y - 1, CONSTANT_K ** iz * initSigma))

                    except (OverflowError, ValueError):
                        # Skip this point if overflow occurs
                        continue
    return keypoints

# (viii) Plot blobs
def plotBlob(img, keypoints, title="Blob detector used Laplacian of Gaussian"):
    fig, axes = plt.subplots()
    axes.set_title(title)
    axes.imshow(img, interpolation='nearest', cmap="gray")
    for blob in keypoints:
        x, y, sigma = blob
        radius = sigma * math.sqrt(2)
        circle = plt.Circle((y, x), radius, color='red', linewidth=1.5, fill=False)
        axes.add_patch(circle)
    axes.set_axis_off()
    plt.show()

# Plot blobs on color images
def plotBlob_color(img_color, keypoints, title="Blob detector used Laplacian of Gaussian"):
    fig, axes = plt.subplots(figsize=(10, 8))
    axes.set_title(title)
    axes.imshow(img_color, interpolation='nearest')
    for blob in keypoints:
        x, y, sigma = blob
        radius = sigma * math.sqrt(2)
        circle = plt.Circle((y, x), radius, color='red', linewidth=2, fill=False)
        axes.add_patch(circle)
    axes.set_axis_off()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Harris corner detector function
def harris_corner_detector(img, k=0.04, threshold=0.01):
    """
    Detect Harris corners in an image
    """
    # Convert to float
    img_float = img.astype(np.float32)

    # Compute derivatives using Sobel
    Ix = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Apply Gaussian smoothing
    Ixx = cv2.GaussianBlur(Ixx, (3, 3), 1.4)
    Iyy = cv2.GaussianBlur(Iyy, (3, 3), 1.4)
    Ixy = cv2.GaussianBlur(Ixy, (3, 3), 1.4)

    # Compute Harris response
    det = Ixx * Iyy - Ixy * Ixy
    trace = Ixx + Iyy
    R = det - k * (trace * trace)

    # Find local maxima
    corners = []
    h, w = R.shape
    for y in range(1, h-1):
        for x in range(1, w-1):
            if R[y, x] > threshold:
                # Check if it's a local maximum
                window = R[y-1:y+2, x-1:x+2]
                if R[y, x] == np.max(window):
                    corners.append((x, y, R[y, x]))

    return corners

# Feature descriptor extraction (simple patch-based)
def extract_patch_descriptor(img, x, y, patch_size=16):
    """
    Extract a simple patch descriptor around a keypoint
    """
    half_size = patch_size // 2
    h, w = img.shape

    # Check bounds
    if (x - half_size < 0 or x + half_size >= w or
        y - half_size < 0 or y + half_size >= h):
        return None

    # Extract patch
    patch = img[y - half_size:y + half_size, x - half_size:x + half_size]

    # Normalize patch
    patch = patch.astype(np.float32)
    patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)

    return patch.flatten()

# Match features between two images
def match_features(corners1, corners2, img1, img2, threshold=0.6):
    """
    Match features between two images using simple correlation
    """
    matches = []

    # Extract descriptors for all corners
    descriptors1 = []
    valid_corners1 = []
    for corner in corners1:
        x, y, response = corner
        desc = extract_patch_descriptor(img1, x, y)
        if desc is not None:
            descriptors1.append(desc)
            valid_corners1.append(corner)

    descriptors2 = []
    valid_corners2 = []
    for corner in corners2:
        x, y, response = corner
        desc = extract_patch_descriptor(img2, x, y)
        if desc is not None:
            descriptors2.append(desc)
            valid_corners2.append(corner)

    # Match descriptors
    for i, desc1 in enumerate(descriptors1):
        best_match_idx = -1
        best_correlation = -1
        second_best = -1

        for j, desc2 in enumerate(descriptors2):
            correlation = np.corrcoef(desc1, desc2)[0, 1]
            if np.isnan(correlation):
                correlation = 0

            if correlation > best_correlation:
                second_best = best_correlation
                best_correlation = correlation
                best_match_idx = j
            elif correlation > second_best:
                second_best = correlation

        # Ratio test
        if best_match_idx >= 0 and best_correlation > threshold:
            if second_best <= 0 or best_correlation / second_best > 1.2:
                matches.append((valid_corners1[i], valid_corners2[best_match_idx]))

    return matches

# Visualize matches
def plot_matches(img1, img2, matches, title="Feature Matches"):
    """
    Plot matches between two images
    """
    # Create side-by-side image
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1
    combined[0:h2, w1:w1+w2] = img2

    # Convert to color for drawing
    combined_color = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined_color)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize matches (color images)
def plot_matches_color(img1_color, img2_color, matches, title="Feature Matches"):
    """
    Plot matches between two color images
    """
    # Create side-by-side image
    h1, w1, _ = img1_color.shape
    h2, w2, _ = img2_color.shape
    h = max(h1, h2)
    w = w1 + w2

    combined = np.zeros((h, w, 3), dtype=np.uint8)
    combined[0:h1, 0:w1] = img1_color
    combined[0:h2, w1:w1+w2] = img2_color

    # Draw matches with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
              (0, 255, 255), (128, 255, 0), (255, 128, 0), (128, 0, 255), (255, 128, 128)]

    plt.figure(figsize=(15, 8))
    plt.imshow(combined)

    for i, match in enumerate(matches):
        corner1, corner2 = match
        x1, y1, _ = corner1
        x2, y2, _ = corner2

        color = colors[i % len(colors)]
        color_norm = np.array(color) / 255.0

        # Draw circles at corner points
        plt.plot(x1, y1, 'o', color=color_norm, markersize=6)
        plt.plot(x2 + w1, y2, 'o', color=color_norm, markersize=6)

        # Draw connecting line
        plt.plot([x1, x2 + w1], [y1, y2], '-', color=color_norm, linewidth=2)

    plt.title(f'{title} ({len(matches)} matches)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()


## TEST RIÊNG CHO PHẦN D: DoG + HARRIS PANORAMA STITCHING


In [ ]:
# ==================== TEST CHỈ PHẦN D ====================
import os

# Các hàm cần thiết cho test (từ implementation phần D)
def keypoints_to_cv2(keypoints):
    """Convert custom keypoints to OpenCV KeyPoints"""
    cv_kps = []
    for kp in keypoints:
        cv_kps.append(cv2.KeyPoint(kp['x'], kp['y'], kp.get('sigma', 5)))
    return cv_kps

def create_test_images_part_d():
    """Tạo ảnh test cho phần D"""
    print("🔧 Tạo ảnh test cho phần D...")

    # Tạo ảnh có đặc trưng rõ ràng để test DoG + Harris
    x, y = np.ogrid[0:200, 0:300]

    # Ảnh 1: Blobs với noise
    img1 = np.zeros((200, 300), dtype=np.float32)
    img1 += 180 * np.exp(-((x - 80)**2 + (y - 80)**2) / (2 * 25**2))
    img1 += 180 * np.exp(-((x - 80)**2 + (y - 220)**2) / (2 * 20**2))
    img1 += 120 * np.exp(-((x - 120)**2 + (y - 150)**2) / (2 * 15**2))

    # Thêm squares cho corners
    img1[60:100, 60:100] = 200
    img1[140:180, 200:240] = 200

    noise1 = np.random.normal(0, 15, img1.shape)
    img1 = np.clip(img1 + noise1, 0, 255).astype(np.uint8)

    # Ảnh 2: Shifted version với overlap
    img2 = np.zeros((200, 300), dtype=np.float32)
    img2 += 180 * np.exp(-((x - 80)**2 + (y - 30)**2) / (2 * 25**2))   # Shift left
    img2 += 180 * np.exp(-((x - 80)**2 + (y - 170)**2) / (2 * 20**2))  # Shift left
    img2 += 120 * np.exp(-((x - 120)**2 + (y - 100)**2) / (2 * 15**2)) # Shift left

    # Shifted squares
    img2[60:100, 10:50] = 200   # Shift left
    img2[140:180, 150:190] = 200 # Shift left

    # Additional features
    img2[40:80, 250:290] = 150

    noise2 = np.random.normal(0, 15, img2.shape)
    img2 = np.clip(img2 + noise2, 0, 255).astype(np.uint8)

    return img1, img2

def test_part_d_complete():
    """Test hoàn chình phần D - DoG + Harris panorama"""
    print("🚀 BẮT ĐẦU TEST PHẦN D - DoG + Harris Panorama Stitching")
    print("="*70)

    # 1. Tạo ảnh test
    img1, img2 = create_test_images_part_d()

    # Hiển thị ảnh test
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].imshow(img1, cmap='gray')
    axes[0].set_title('Test Image 1')
    axes[0].axis('off')
    axes[1].imshow(img2, cmap='gray')
    axes[1].set_title('Test Image 2')
    axes[1].axis('off')
    plt.suptitle('📸 Ảnh Test cho Phần D', fontsize=14)
    plt.tight_layout()
    plt.show()

    # 2. Xây dựng pyramids
    print("\n📊 BƯỚC 1: Xây dựng Gaussian và DoG Pyramids")
    gauss1 = build_gaussian_pyramid(img1, num_octaves=3, s=3, sigma0=1.6)
    gauss2 = build_gaussian_pyramid(img2, num_octaves=3, s=3, sigma0=1.6)

    dog1 = build_dog_pyramid(gauss1)
    dog2 = build_dog_pyramid(gauss2)

    print(f"✅ Gaussian pyramid 1: {len(gauss1)} octaves")
    print(f"✅ Gaussian pyramid 2: {len(gauss2)} octaves")
    print(f"✅ DoG pyramid 1: {len(dog1)} octaves")
    print(f"✅ DoG pyramid 2: {len(dog2)} octaves")

    # 3. Phát hiện keypoints
    print("\n🎯 BƯỚC 2: Phát hiện DoG + Harris Keypoints")
    kps1 = find_keypoints(gauss1, dog1, s=3, threshold=0.05, border=5)
    kps2 = find_keypoints(gauss2, dog2, s=3, threshold=0.05, border=5)

    print(f"📍 Keypoints trước NMS - Ảnh 1: {len(kps1)}, Ảnh 2: {len(kps2)}")

    # 4. Non-Maximum Suppression
    kps1_nms = nonmax_suppression(kps1, radius=8)
    kps2_nms = nonmax_suppression(kps2, radius=8)

    print(f"📍 Keypoints sau NMS - Ảnh 1: {len(kps1_nms)}, Ảnh 2: {len(kps2_nms)}")

    if len(kps1_nms) == 0 or len(kps2_nms) == 0:
        print("❌ Không tìm thấy keypoints! Test thất bại.")
        return False

    # 5. Hiển thị keypoints
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Ảnh 1 với keypoints
    axes[0].imshow(img1, cmap='gray')
    axes[0].set_title(f'🎯 Keypoints Ảnh 1 ({len(kps1_nms)} points)')
    for i, kp in enumerate(kps1_nms[:15]):  # Hiển thị tối đa 15 keypoints
        x, y = kp[1], kp[0]  # Swap vì kp format là (y, x, octave, sigma, response)
        sigma = kp[3] if len(kp) > 3 else 5
        circle = plt.Circle((x, y), sigma, color='red', fill=False, linewidth=2)
        axes[0].add_patch(circle)
        axes[0].plot(x, y, 'ro', markersize=4)
        axes[0].text(x+2, y-2, str(i), color='yellow', fontsize=8)
    axes[0].axis('off')

    # Ảnh 2 với keypoints
    axes[1].imshow(img2, cmap='gray')
    axes[1].set_title(f'🎯 Keypoints Ảnh 2 ({len(kps2_nms)} points)')
    for i, kp in enumerate(kps2_nms[:15]):
        x, y = kp[1], kp[0]
        sigma = kp[3] if len(kp) > 3 else 5
        circle = plt.Circle((x, y), sigma, color='red', fill=False, linewidth=2)
        axes[1].add_patch(circle)
        axes[1].plot(x, y, 'ro', markersize=4)
        axes[1].text(x+2, y-2, str(i), color='yellow', fontsize=8)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

    # 6. Chuyển đổi sang OpenCV keypoints
    print("\n🔄 BƯỚC 3: Chuyển đổi và tính Descriptors")
    kps1_cv = kps_to_cv2_keypoints(kps1_nms)
    kps2_cv = kps_to_cv2_keypoints(kps2_nms)

    # 7. Tính descriptors
    orb = cv2.ORB_create(nfeatures=500)
    _, des1 = orb.compute(img1, kps1_cv)
    _, des2 = orb.compute(img2, kps2_cv)

    if des1 is None or des2 is None:
        print("❌ Không tính được descriptors! Test thất bại.")
        return False

    print(f"📝 Descriptors - Ảnh 1: {len(des1)}, Ảnh 2: {len(des2)}")

    # 8. Matching
    print("\n🔗 BƯỚC 4: Feature Matching")
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    # Ratio test
    good_matches = []
    for match_pair in matches:
        if len(match_pair) == 2:
            m, n = match_pair
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

    print(f"🎯 Good matches: {len(good_matches)}")

    if len(good_matches) < 4:
        print("❌ Không đủ matches để ước lượng homography!")
        return False

    # 9. Hiển thị matches
    img_matches = cv2.drawMatches(img1, kps1_cv, img2, kps2_cv, good_matches[:20], None,
                                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.imshow(img_matches, cmap='gray')
    plt.title('Corresponding Keypoints with Lines')
    plt.show()

    # 10. Ước lượng Homography
    print("\n📐 BƯỚC 5: Ước lượng Homography")
    if len(good_matches) >= 4:
        pts1 = np.float32([kps1_cv[m.queryIdx].pt for m in good_matches])
        pts2 = np.float32([kps2_cv[m.trainIdx].pt for m in good_matches])

        H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC, 3.0)

        if H is not None:
            inliers = np.sum(mask)
            print(f"✅ Homography ước lượng thành công!")
            print(f"📊 Inliers: {inliers}/{len(good_matches)} = {inliers/len(good_matches)*100:.1f}%")

            # 11. Tạo Panorama
            print("\n🖼️  BƯỚC 6: Tạo Panorama")
            pano = warp_and_blend(img1, img2, H)

            # Hiển thị kết quả cuối cùng
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))

            axes[0,0].imshow(img1, cmap='gray')
            axes[0,0].set_title('📸 Ảnh gốc 1')
            axes[0,0].axis('off')

            axes[0,1].imshow(img2, cmap='gray')
            axes[0,1].set_title('📸 Ảnh gốc 2')
            axes[0,1].axis('off')

            axes[1,0].imshow(img_matches, cmap='gray')
            axes[1,0].set_title(f'🔗 Matches ({len(good_matches)})')
            axes[1,0].axis('off')

            axes[1,1].imshow(pano, cmap='gray')
            axes[1,1].set_title('🎉 KẾT QUẢ PANORAMA')
            axes[1,1].axis('off')

            plt.suptitle('📊 TỔNG KẾT TEST PHẦN D - DoG + Harris Panorama', fontsize=16)
            plt.tight_layout()
            plt.show()

            print("\n" + "="*70)
            print("🎉 TEST PHẦN D HOÀN THÀNH THÀNH CÔNG!")
            print("✅ Xây dựng Gaussian & DoG pyramids: OK")
            print("✅ Phát hiện DoG keypoints: OK")
            print("✅ Lọc Harris corner response: OK")
            print("✅ Feature matching: OK")
            print("✅ Homography estimation: OK")
            print("✅ Panorama stitching: OK")
            print("="*70)

            return True
        else:
            print("❌ Thất bại trong việc ước lượng homography!")
            return False
    else:
        print("❌ Không đủ matches!")
        return False


In [ ]:
# Chạy test phần D
test_result = test_part_d_complete()

if test_result:
    print("\n🏆 PHẦN D HOẠT ĐỘNG HOÀN HẢO!")
    print("Pipeline DoG + Harris keypoint detection và panorama stitching đã được triển khai thành công.")
else:
    print("\n⚠️ PHẦN D CẦN KIỂM TRA LẠI!")
    print("Có vấn đề trong pipeline, cần debug thêm.")


## KIỂM TRA VỚI ẢNH THẬT (NẾU CÓ)


In [ ]:
# Test với ảnh thật nếu có sẵn
def test_part_d_real_images():
    """Test phần D với ảnh thật"""
    print("🔍 Tìm ảnh thật để test phần D...")

    # Danh sách ảnh có thể test
    test_pairs = [
        ("book1.jpg", "book2.jpg"),
        ("t1.jpg", "test2.png"),
    ]

    lab1_path = "E:/Pycharm/Advanced-Reading-on-Computer-Vision/Lab1/"

    for img1_name, img2_name in test_pairs:
        img1_path = os.path.join(lab1_path, img1_name)
        img2_path = os.path.join(lab1_path, img2_name)

        if os.path.exists(img1_path) and os.path.exists(img2_path):
            print(f"📁 Tìm thấy: {img1_name} và {img2_name}")

            img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
            img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

            if img1 is not None and img2 is not None:
                # Resize nếu cần
                max_size = 500
                if img1.shape[0] > max_size or img1.shape[1] > max_size:
                    scale = max_size / max(img1.shape)
                    img1 = cv2.resize(img1, None, fx=scale, fy=scale)

                if img2.shape[0] > max_size or img2.shape[1] > max_size:
                    scale = max_size / max(img2.shape)
                    img2 = cv2.resize(img2, None, fx=scale, fy=scale)

                print(f"📏 Kích thước: {img1.shape} và {img2.shape}")

                # Test với ảnh thật
                print(f"\n🎯 TEST PHẦN D VỚI ẢNH THẬT: {img1_name} + {img2_name}")

                # Sử dụng parameters phù hợp với ảnh thật
                try:
                    gauss1 = build_gaussian_pyramid(img1, num_octaves=3, s=3)
                    gauss2 = build_gaussian_pyramid(img2, num_octaves=3, s=3)

                    dog1 = build_dog_pyramid(gauss1)
                    dog2 = build_dog_pyramid(gauss2)

                    kps1 = find_keypoints(gauss1, dog1, threshold=0.02)
                    kps2 = find_keypoints(gauss2, dog2, threshold=0.02)

                    if len(kps1) > 0 and len(kps2) > 0:
                        kps1_nms = nonmax_suppression(kps1, radius=10)
                        kps2_nms = nonmax_suppression(kps2, radius=10)

                        print(f"✅ Keypoints: {len(kps1_nms)} và {len(kps2_nms)}")

                        # Hiển thị kết quả
                        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
                        axes[0].imshow(img1, cmap='gray')
                        axes[0].set_title(f'{img1_name} ({len(kps1_nms)} keypoints)')

                        # Vẽ keypoints
                        for kp in kps1_nms[:20]:
                            x, y = kp[1], kp[0]
                            axes[0].plot(x, y, 'ro', markersize=3)
                        axes[0].axis('off')

                        axes[1].imshow(img2, cmap='gray')
                        axes[1].set_title(f'{img2_name} ({len(kps2_nms)} keypoints)')

                        for kp in kps2_nms[:20]:
                            x, y = kp[1], kp[0]
                            axes[1].plot(x, y, 'ro', markersize=3)
                        axes[1].axis('off')

                        plt.suptitle('🎯 Test Phần D với Ảnh Thật')
                        plt.tight_layout()
                        plt.show()

                        print("✅ Test với ảnh thật thành công!")
                        return True

                    else:
                        print("❌ Không tìm thấy keypoints trong ảnh thật")

                except Exception as e:
                    print(f"❌ Lỗi khi test với ảnh thật: {e}")

    print("❌ Không tìm thấy ảnh thật phù hợp để test")
    return False

# Chạy test với ảnh thật
real_test_result = test_part_d_real_images()

if real_test_result:
    print("\n🎊 PHẦN D HOẠT ĐỘNG TỐT VỚI CẢ ẢNH THẬT!")
else:
    print("\n📝 PHẦN D chỉ test được với ảnh synthetic (vẫn OK!)")

print("\n" + "🎯"*30)
print("TEST PHẦN D HOÀN TẤT!")
print("🎯"*30)
